In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
chat = ChatOpenAI(
    temperature = 0.1
)
template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}? Also what is your name?"
)
prompt=template.format(country_a="USA", country_b="Canada")

chat.predict(prompt)


'The distance between the United States and Canada varies depending on the specific locations being compared. The shortest distance between the two countries is approximately 8 kilometers (5 miles) between the Northwest Angle Inlet in Minnesota, USA, and the Elm Point in Manitoba, Canada. However, if we consider the distance between major cities, for example, New York City, USA, and Toronto, Canada, the distance is approximately 550 kilometers (340 miles).\n\nAs an AI language model developed by OpenAI, I don\'t have a personal name. You can simply refer to me as "ChatGPT." How can I assist you today?'

In [21]:

template = ChatPromptTemplate.from_messages([
    ("system","You are a geography expert. And you only reply in {language}."
        ),
    ("ai","Ciao! mi chiamo {name}!"
        ),
    ("human","What is the distance between {country_a} and {country_b}? Also what is your name?")
    
])

prompt=template.format_messages(language="Korean", name="Giovanni", country_a="USA", country_b="Canada")

chat.predict_messages(prompt)

AIMessage(content='안녕하세요! 저는 지리 전문가입니다. 미국과 캐나다 사이의 거리는 다양합니다. 예를 들어, 뉴욕과 토론토 사이의 거리는 약 550 킬로미터입니다. 제 이름은 OpenAI Assistant입니다. 무엇을 도와드릴까요?')

In [22]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommaOutputParser()
p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

In [27]:
template = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a list generating machine. Everyting you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else.",
        ),
        ("human","{question}"),
    ]
)



In [29]:
chain = template | chat | CommaOutputParser()

chain.invoke({"max_items":5, "question":"What are the pokemons?"})

['Pikachu', 'Charizard', 'Bulbasaur', 'Squirtle', 'Jigglypuff']